In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import urllib
import traceback

In [2]:
data_test=pd.read_csv("SNLP_2020_test.nt",sep=" ",names=["1","2","3","."])

In [3]:
data_test.head()


,1,2,3,.
0,<http://swc2017.aksw.org/task2/dataset/3417193>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#St...,.
1,<http://swc2017.aksw.org/task2/dataset/3417193>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#su...,<http://dbpedia.org/resource/Philip_Wylie>,.
2,<http://swc2017.aksw.org/task2/dataset/3417193>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#pr...,<http://dbpedia.org/ontology/author>,.
3,<http://swc2017.aksw.org/task2/dataset/3417193>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ob...,<http://dbpedia.org/resource/When_Worlds_Collide>,.
4,<http://swc2017.aksw.org/task2/dataset/3812648>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#St...,.


In [4]:
def check(s,o,sparql):
    """
    This function checks, if object could be found in "abstract" and "wikiPageWikiLinks" of subject.
    Also if there is an redirection to other page for same subject, the function will recognize it.
    """
    punc = '''!()-[]{};:'"\,<>.?@#$%^&*_~'''

    sparql.setQuery(
        "SELECT ?a WHERE {{\
        "+s+" <http://dbpedia.org/ontology/abstract> ?a FILTER (  langMatches(lang(?a),'en') ) .}UNION{\
        "+s+" <http://dbpedia.org/ontology/wikiPageWikiLink> ?a .\
        }}")
    sparql.setReturnFormat(JSON)
    try:
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        if len(results["results"]["bindings"])==1:
            s="<"+results["results"]["bindings"][0]["a"]["value"]+">"
            sparql.setQuery(
                "SELECT ?a WHERE {{\
                "+s+" <http://dbpedia.org/ontology/abstract> ?a FILTER (  langMatches(lang(?a),'en') ) .}UNION{\
                "+s+" <http://dbpedia.org/ontology/wikiPageWikiLink> ?a .\
                }}")
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
        
        string_to_search=""
        string_to_find=urllib.parse.unquote(o)
        string_to_find=string_to_find.split("/")[-1][:-1].replace("_"," ")
        for ele in punc: 
            string_to_find = string_to_find.replace(ele, " ") 
        for row in results["results"]["bindings"]:
            if row["a"]["type"]=="uri":
                value=row["a"]["value"].split("/")[-1][:].replace("_"," ").lower()
            if row["a"]["type"]=="literal":
                value=row["a"]["value"].replace("_"," ").lower()        
            for ele in punc: 
                value = value.replace(ele, "") 
            string_to_search+=" "+value
    except:
        print("error")
        print(s,o)
        return "0.0"
    string_to_search=string_to_search.lower().split(" ")
    string_to_find=string_to_find.lower().split(" ")
    
    if all(item in string_to_search for item in string_to_find):
        return "1.0"

    else:
        return "0.0"
    

In [5]:
def check2(s,p,o,sparql):
    """
    This function checks, if object could be found in "abstract" and "wikiPageWikiLinks" of subject.
    Also if there is an redirection to other page for same subject, the function will recognize it.
    
    In difference to the "check" function this one also exams, if there is the property in the abstract.
    And if the property is there, and object is not to find withiin +- 10 tokens, then respond is '0.0'.
    At the end we recognized, that there is no any advantages of this function on that exactly data set , 
    but "do not touch running system if you don't need to"
    """
    punc = '''!()-[]{};:'"\,<>.?@#$%^&*_~'''
    abstract=None
    sparql.setQuery(
        "SELECT ?a WHERE {{\
        "+s+" <http://dbpedia.org/ontology/abstract> ?a FILTER (  langMatches(lang(?a),'en') ) .}UNION{\
        "+s+" <http://dbpedia.org/ontology/wikiPageWikiLink> ?a .\
        }}")
    sparql.setReturnFormat(JSON)
    try:
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        if len(results["results"]["bindings"])==1:
            s="<"+results["results"]["bindings"][0]["a"]["value"]+">"
            sparql.setQuery(
                "SELECT ?a WHERE {{\
                "+s+" <http://dbpedia.org/ontology/abstract> ?a FILTER (  langMatches(lang(?a),'en') ) .}UNION{\
                "+s+" <http://dbpedia.org/ontology/wikiPageWikiLink> ?a .\
                }}")
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
        
        string_to_search=""
        string_to_find=urllib.parse.unquote(o)
        string_to_find=string_to_find.split("/")[-1][:-1].replace("_"," ")
        prop=p.split("/")[-1][:-1]
        for ele in punc: 
            string_to_find = string_to_find.replace(ele, " ") 
        for row in results["results"]["bindings"]:
            if row["a"]["type"]=="uri":
                value=row["a"]["value"].split("/")[-1][:].replace("_"," ").lower()
            if row["a"]["type"]=="literal":
                value=row["a"]["value"].replace("_"," ").lower()
                abstract=value
            for ele in punc: 
                value = value.replace(ele, "") 
            string_to_search+=" "+value
    except:
        traceback.print_exc()
        print("error")
        print(s,o)
        return "0.0"
    string_to_search=string_to_search.lower().split(" ")
    string_to_find=string_to_find.lower().split(" ")
    #abstract=abstract.split(" ")
    if all(item in string_to_search for item in string_to_find):
        return "1.0"
    
    
        # this is exactly the difference between check and check2.
        if prop in string_to_search:
            for i,token in enumerate(string_to_search):
                if token==prop:
                    if all(item in string_to_search[i-10:i+10] for item in string_to_find):
                        return "1.0"
                    else:
                        return "0.0"
        else:
            return "1.0"
    else:
        return "0.0"
    

In [6]:
range_list=["<http://dbpedia.org/ontology/subsidiary>",
           "<http://dbpedia.org/ontology/award>",
           "<http://dbpedia.org/ontology/deathPlace>",
           "<http://dbpedia.org/ontology/spouse>",
           "<http://dbpedia.org/ontology/subsidiary>",
           "<http://dbpedia.org/ontology/starring>",
           "<http://dbpedia.org/ontology/foundationPlace>",
           "<http://dbpedia.org/ontology/author>"]
ignore_list=["<http://dbpedia.org/ontology/team>"]
def check_property(p,o,sparql):
    """
    This function checks if object type suits for property range. 
    If not, the functon returns "0.0".
    Some cases are defined separatly.
    """
    if p in range_list:
        sparql.setQuery(
                "SELECT ?a ?b WHERE {{\
                "+p+" <http://www.w3.org/2000/01/rdf-schema#range> ?a .}UNION{\
                "+o+" <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?b.\
                }}")   
    else:
        return "1.0"    
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    a=results["results"]["bindings"][0]["a"]["value"]
    b=[]
    for row in results["results"]["bindings"]:
        try:
            b.append(row["b"]["value"])
        except:
            continue
    if a in b:
        return "1.0"
    elif p=="<http://dbpedia.org/ontology/starring>" and "http://dbpedia.org/ontology/Person" in b:
        return "1.0"
    elif p=="<http://dbpedia.org/ontology/team>" and "http://dbpedia.org/ontology/Organisation" in b:
        return "1.0"
    elif p=="<http://dbpedia.org/ontology/foundationPlace>" and "http://dbpedia.org/ontology/Location" in b:
        return "1.0"
    elif p=="<http://dbpedia.org/ontology/author>" and "http://dbpedia.org/ontology/Work" in b:
        return "1.0"
    else:
        return "0.0"

In [7]:
s=None
o=None
p=None
answer=None
flag=False
dataset_id=None

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

with open('submit.nt', 'w') as f:

    
    for i,row in data_test.iterrows():
        if i%200==0:
            print(i)
        if row["3"]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement>":
            s=None
            o=None
            p=None
            answer=None
            flag=False
            dataset_id=None
            continue 

        
        if row["2"]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#subject>":
            s=row["3"]
            dataset_id=row["1"]
            continue
         
        if row["2"]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#object>":
            o=row["3"]
            flag=True
        if row["2"]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate>":
            p=row["3"]
            
        if flag==True:
            # check if Abstract and wikilings contain object. If yes, then answer is "1.0"
            direkt_answer=check2(s,p,o,sparql)
            # check if Abstract and wikilings of object contain subject. If yes, then answer is "1.0"
            backword_answer=check(o,s,sparql)
            # if one of the above answers is 1.0, check if object type is in range of property
            if direkt_answer=="1.0" or backword_answer=="1.0":
                my_answer=check_property(p,o,sparql)
            else:
                my_answer="0.0"
            line=dataset_id+ " "+"<http://swc2017.aksw.org/hasTruthValue> "+'"'+my_answer+'"'+"^^<http://www.w3.org/2001/XMLSchema#double> .\n"
            f.write(line)
    f.close()

0
200
400
600


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/SPARQLWrapper/Wrapper.py", line 1073, in _query
    response = urlopener(request)
  File "/usr/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/usr/lib/python3.8/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/usr/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.8/urllib/request.py", line 755, in http_error_302
    return self.parent.open(new, timeout=req.timeout)
  File "/usr/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.8/urllib/request.py", line 640, in http_response
    response =

error
<OpenLink Software, Inc., is a software company founded in 1992 with its headquarters in Burlington, Massachusetts, United States, although some research and product development takes place within the EU, across the UK, Netherlands and Bulgaria. The company develops and deploys standards-compliant middleware products that cover: 
*  Transparent access to SQL data sources via ODBC and JDBC drivers, and OLE-DB, ADO.NET, and XMLA data providers; 
*  Enterprise Data Integration of heterogeneous SQL and XML and RDF data sources; 
*  Web Services Composition and Deployment; 
*  Business Process Management and Integration. OpenLink Software is creator and owner of the Universal Data Access drivers suite(comprising ODBC drivers, JDBC drivers, OLE-DB providers, ADO.NET providers, and XMLA providers); the Virtuoso Universal Server; the iODBC driver manager; an AJAX toolkit for rich Internet application development; OpenLink Data Spaces; and other middleware products. Other sources of reven

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/SPARQLWrapper/Wrapper.py", line 1073, in _query
    response = urlopener(request)
  File "/usr/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/usr/lib/python3.8/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/usr/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.8/urllib/request.py", line 755, in http_error_302
    return self.parent.open(new, timeout=req.timeout)
  File "/usr/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.8/urllib/request.py", line 640, in http_response
    response =

error
<LogiSense is a privately owned software development company headquartered in Cambridge, Ontario Canada. LogiSense's Billing Solution provides Telecommunications billing, Operational Support Systems, Captive Portal and Bandwidth management solutions. Traditionally LogiSense services growth IP markets with customers including Digiweb, Infosat Communications, SimpleSignal, Atria Networks (acquired by Rogers Communications), Telesphere, KMTS (acquired by Bell Aliant) among others.> <http://dbpedia.org/resource/Ontario>
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
